In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# ==============================================
# 【1. 対象フォルダ内のCSVファイルを探索・取得】
# スクリプトと同じディレクトリ内の全てのCSVファイルを取得します。
# ==============================================
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# ==============================================
# 【2. カラーバーの表示範囲設定】
# 各物性値ごとにカラーバー（カラーマップ）の最小値・最大値を指定します。
# ==============================================
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 600),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 6),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

# ==============================================
# 【3. ファイルごとに処理ループ】
# 各CSVファイルについて以下の処理を行います。
# ==============================================
for file_path in csv_files:
    print(f"読み込み中: {file_path}")

    # Shift-JIS で読み込み（Excel 日本語CSV対応）
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # ==========================================
    # 【3-1. 対象となる物性値の名称を取得】
    # 1行目から対象とする物性値名を取得します。
    # ==========================================
    full_name = str(df.iloc[0,0])
    if "/" in full_name:
        target_name = full_name.split("/", 1)[1].strip()
    else:
        target_name = full_name.strip()
    print(f"対象値: {target_name}")

    # ==========================================
    # 【3-2. X(Y)軸値と測定値（Z値）の抽出】
    # Coil（X軸）, Platen（Y軸）, 測定値（Z値）をデータから抽出します。
    # ==========================================
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])
    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # ==========================================
    # 【3-3. 2次の多項式回帰によるフィッティング】
    # Coil, Platenを説明変数とし、測定値を2次多項式で近似します。
    # ==========================================
    X = np.column_stack((x, y))
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, z)

    # ==========================================
    # 【3-4. グリッド状の座標を作成し回帰式で予測】
    # プロット用に等間隔グリッドを作成し、多項式モデルで値を予測します。
    # ==========================================
    xi = np.linspace(0, 1500, 200)
    yi = np.linspace(0, 400, 200)
    Xi, Yi = np.meshgrid(xi, yi)
    XY_poly = poly.transform(np.column_stack((Xi.ravel(), Yi.ravel())))
    Zi = model.predict(XY_poly).reshape(Xi.shape)

    # ==========================================
    # 【3-5. 下限値を0に制限（Vdcは除外）】
    # Vdc以外の値は負の値を0にクリップします。
    # ==========================================
    if target_name != "Vdc":
        Zi = np.clip(Zi, 0, None)

    # ==========================================
    # 【3-6. カラーバーの範囲（vmin, vmax）の設定】
    # 物性値ごとに色の範囲を設定します。設定がなければ自動。
    # ==========================================
    vmin, vmax = color_ranges.get(target_name, (None, None))

    # ==========================================
    # 【3-7. 回帰式（数式）の生成】
    # 係数や項を取得し、グラフに表示するためLaTeX形式の式を作ります。
    # ==========================================
    coef = model.coef_
    intercept = model.intercept_
    feature_names = poly.get_feature_names_out(['X','Y'])
    equation = rf"$Z = {intercept:.5f}"
    for i, c in enumerate(coef):
        if c >= 0:
            sign = " + "
        else:
            sign = " - "
            c = abs(c)
        term = feature_names[i].replace(" "," \\cdot ").replace("^2","^{2}")
        equation += f"{sign}{c:.5f} {term}"
    equation += "$"

    # ==========================================
    # 【3-8. 描画と保存】
    # 回帰面（等高線）、測定点（散布図）、カラーバー、回帰式などを図に表示し、PNG画像として保存します。
    # ==========================================
    plt.figure(figsize=(10,8))
    plt.rcParams.update({'font.size': 18})
    cp = plt.contourf(Xi, Yi, Zi, levels=20, cmap="viridis", vmin=vmin, vmax=vmax)

    # --- カラーバー目盛りをきりのいい数字に設定 ---
    if vmin is not None and vmax is not None:
        if target_name == "Mask Selectivity ratio":
            ticks = np.arange(vmin, vmax+0.1, 1.0)
        elif target_name == "Vdc":
            ticks = np.arange(vmin, vmax+200, 200)
        else:
            step = 100 if vmax > 100 else 10
            ticks = np.arange(vmin, vmax+step, step)
        cbar = plt.colorbar(cp, label=target_name, ticks=ticks)
    else:
        cbar = plt.colorbar(cp, label=target_name)

    import matplotlib.colors as mcolors

    # --- Normalizeを作成（共通化） ---
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

    # --- 等高線プロット ---
    cp = plt.contourf(Xi, Yi, Zi, levels=20, cmap="viridis", norm=norm)

    # --- 散布図プロット（測定値に色付け） ---
    sc = plt.scatter(x, y, c=z, cmap="viridis", norm=norm,
                 edgecolors="black", linewidths=0.5, s=80)

    # --- ラベル・タイトル・回帰式 ---
    plt.xlabel("Coil [W]")
    plt.ylabel("Platen [W]")
    plt.title(f"Predicted {target_name} (Polynomial Fit)")
    plt.text(0.05, 0.95, equation, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7))

    # --- ファイル保存 ---
    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    output_file = f"{safe_name}_prediction.png"
    output_path = os.path.join(script_dir, output_file)
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"グラフを保存しました: {output_path}")